In [1]:
from pyspark.sql import SQLContext

In [2]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local[2]"))

In [3]:
sqlContext=SQLContext(sc)

In [5]:
df=sqlContext.read.load("daily_weather.csv",format="com.databricks.spark.csv",header='true',inferSchema="true")

In [7]:
df.columns

['number',
 'air_pressure_9am',
 'air_temp_9am',
 'avg_wind_direction_9am',
 'avg_wind_speed_9am',
 'max_wind_direction_9am',
 'max_wind_speed_9am',
 'rain_accumulation_9am',
 'rain_duration_9am',
 'relative_humidity_9am',
 'relative_humidity_3pm']

In [8]:
df.printSchema()

root
 |-- number: integer (nullable = true)
 |-- air_pressure_9am: double (nullable = true)
 |-- air_temp_9am: double (nullable = true)
 |-- avg_wind_direction_9am: double (nullable = true)
 |-- avg_wind_speed_9am: double (nullable = true)
 |-- max_wind_direction_9am: double (nullable = true)
 |-- max_wind_speed_9am: double (nullable = true)
 |-- rain_accumulation_9am: double (nullable = true)
 |-- rain_duration_9am: double (nullable = true)
 |-- relative_humidity_9am: double (nullable = true)
 |-- relative_humidity_3pm: double (nullable = true)



In [9]:
df.describe().show()

+-------+------------------+-----------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+------------------+---------------------+---------------------+
|summary|            number| air_pressure_9am|      air_temp_9am|avg_wind_direction_9am|avg_wind_speed_9am|max_wind_direction_9am|max_wind_speed_9am|rain_accumulation_9am| rain_duration_9am|relative_humidity_9am|relative_humidity_3pm|
+-------+------------------+-----------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+------------------+---------------------+---------------------+
|  count|              1095|             1092|              1090|                  1091|              1092|                  1092|              1091|                 1089|              1092|                 1095|                 1095|
|   mean|             547.0|918.8825513138094| 64.9330014128

In [10]:
df.describe("air_pressure_9am").show()

+-------+-----------------+
|summary| air_pressure_9am|
+-------+-----------------+
|  count|             1092|
|   mean|918.8825513138094|
| stddev|3.184161180386833|
|    min|907.9900000000024|
|    max|929.3200000000012|
+-------+-----------------+



In [11]:
len(df.columns)

11

In [12]:
df.count()

1095

In [13]:
# Drop Missing Values in air_pressure_9am column
df2=df.na.drop(subset=['air_pressure_9am'])

In [14]:
df2.count()

1092

In [15]:
df2.columns

['number',
 'air_pressure_9am',
 'air_temp_9am',
 'avg_wind_direction_9am',
 'avg_wind_speed_9am',
 'max_wind_direction_9am',
 'max_wind_speed_9am',
 'rain_accumulation_9am',
 'rain_duration_9am',
 'relative_humidity_9am',
 'relative_humidity_3pm']

In [16]:
df2.describe().show()

+-------+-----------------+-----------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+------------------+---------------------+---------------------+
|summary|           number| air_pressure_9am|      air_temp_9am|avg_wind_direction_9am|avg_wind_speed_9am|max_wind_direction_9am|max_wind_speed_9am|rain_accumulation_9am| rain_duration_9am|relative_humidity_9am|relative_humidity_3pm|
+-------+-----------------+-----------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+------------------+---------------------+---------------------+
|  count|             1092|             1092|              1087|                  1088|              1089|                  1089|              1088|                 1086|              1089|                 1092|                 1092|
|   mean|546.8324175824176|918.8825513138094| 64.96896753146552|

In [17]:
df2.stat.corr('rain_accumulation_9am','rain_duration_9am')

0.7298253479609021

In [18]:
################################# Assigment ###########################

In [19]:
# What is the maximum of the average wind speed measurements at 9am (to 2 decimal places)?

df.describe(['avg_wind_speed_9am']).show()


+-------+------------------+
|summary|avg_wind_speed_9am|
+-------+------------------+
|  count|              1092|
|   mean|  5.50828424225493|
| stddev|4.5528134655317185|
|    min|  0.69345139999974|
|    max|23.554978199999763|
+-------+------------------+



In [20]:
# How many rows containing rain accumulation at 9am measurements have missing values?

df.describe(['rain_accumulation_9am']).show()

# Answer 1095-1089=6

+-------+---------------------+
|summary|rain_accumulation_9am|
+-------+---------------------+
|  count|                 1089|
|   mean|  0.20307895225211126|
| stddev|   1.5939521253574893|
|    min|                  0.0|
|    max|    24.01999999999907|
+-------+---------------------+



In [21]:
# What is the correlation between the relative humidity at 9am and at 3pm (to 2 decimal places, and without removing or imputing missing values)?

df.stat.corr('relative_humidity_9am','relative_humidity_3pm')

0.8830741909793918

In [22]:
# If the histogram for air temperature at 9am has 50 bins, what is the number of elements in the bin with the most elements (without removing or imputing missing values)?


df.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1090|
|   mean| 64.93300141287072|
| stddev|11.175514003175877|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



In [23]:
df.stat.corr('max_wind_speed_9am','max_wind_direction_9am')


-0.3779988368102176

In [24]:
df.describe(['max_wind_speed_9am']).show()

+-------+------------------+
|summary|max_wind_speed_9am|
+-------+------------------+
|  count|              1091|
|   mean| 7.019513529175272|
| stddev| 5.598209170780958|
|    min|1.1855782000000479|
|    max| 29.84077959999996|
+-------+------------------+



In [ ]:
################# Data Exploration Using SQL Quaries ###############################

In [25]:
df.registerTempTable('weather')

In [27]:
def getMaxTemp():
    return sqlContext.sql('SELECT MAX(air_temp_9am) as maxtemp FROM weather').first().maxtemp

print(getMaxTemp())
    

98.90599999999992


In [28]:
def getAceragePressure():
    return sqlContext.sql('SELECT AVG(air_pressure_9am) as avgPressure FROM weather').first().avgPressure

print(getAceragePressure())

918.8825513138094
